# Desafio técnico Charla
**Autor**: Paulo Rohan

Esse notebook contém a resolução do desafio técnico para a vaga de **Jr AI Engineer** da startup Charla.
A descrição do desafio é a seguinte:

"Você deve criar um script em Python que utiliza a biblioteca PydanticAI para criar um agente de IA que extraia informações específicas de uma Nota Fiscal de Serviço (PDF) que forneceremos. O objetivo é ler o conteúdo do PDF, usar um LLM para “entender” o texto e retornar os dados de forma estruturada e validada."

### 1. Configuração do ambiente

In [10]:
# Carregando os pacotes necessários

## Manipulação de dados e arquivos
from pathlib import Path
from datetime import date
import json

## Anotação de erros e logging
import logging

## Definição de modelos e validação
from pydantic import BaseModel, Field

## Construção do agente
from pydantic_ai import Agent, BinaryContent
from pydantic_ai.models.google import GoogleModel
from pydantic_ai.providers.google import GoogleProvider
from google.oauth2 import service_account

## Execução assíncrona
import asyncio

### 2. Leitura do PDF

Embora a tarefa pudesse ser realizada com apenas um único arquivo PDF refente a Nota Fiscal de Serviço, optei por utilizar dois arquivos. A minha escolha foi principalmente para tratar a tarefa de uma maneira mais escalável, construíndo a solução de forma que pudesse lidar com múltiplos arquivos. Dessa forma, busquei listar todos os arquivos PDF que estariam contidos dentro da pasta de armazenamento, utilizando a biblioteca `Path`, e em seguida obter o seu caminho, para que pudesse ser utilizado na abordagem de leitura com `BinaryContent`. Nessa etapa, apenas o caminho está sendo armazenado, com a leitura sendo feita posteriormente, no momento da execução da extração.

In [11]:
# Extraíndo os arquivos PDF contidos na pasta dados
pasta_dados = Path('dados')

# Armazenando os arquivos em uma lista
lista_pdfs = [str(arquivo) for arquivo in pasta_dados.glob('*.pdf')]

# Verificando os arquivos encontrados
print("Arquivos encontrados:\n")
for arquivo in lista_pdfs:
    print(arquivo)

Arquivos encontrados:

dados\nf_paulo_rohan.pdf
dados\nf_paulo_rohan_2.pdf


### 3. Modelagem dos dados

O próximo passo foi utilizar a biblioteca `Pydantic` para representar a estrutura da informação que será extraída a partir da Nota Fiscal de Serviço. O `Pydantic`tem a função de definir modelos de dados com validação automática, garantindo que os dados extraídos sejam consistentes e estejam no formato correto. Os itens que devem ser extraídos são os seguintes:

1. Descrição do Serviço
2. Valor do serviço
3. Número da Nota
4. Data de emissão
5. Valor total
6. CNPJ do Prestador de Serviço

Com base nisso, o tipo apropriado dos dados foi indicado na definição da estrutura para validação.

In [12]:
# Definindo o Pydantic com o modelo da estrutura de extração
class ExtracaoOutput(BaseModel):
    """
    Define a estrutura de informação a ser extraída da nota fiscal.
    """
    descricao_servico: str = Field(description="Descrição detalhada do serviço prestado.")
    valor_servico: float = Field(description="Valor referente ao item de serviço.")
    numero_nota: str = Field(description="Número identificador da nota fiscal.")
    data_emissao: date = Field(description="Data de emissão da nota fiscal.")
    valor_total: float = Field(description="Valor total líquido da nota fiscal.")
    cnpj_prestador: str = Field(description="CNPJ (Cadastro Nacional da Pessoa Jurídica) da empresa prestadora do serviço.")


### 4. Extração de informações com pyndaticAI

Finalmente, utilizando as credenciais fornecidas, o modelo Gemini 2.5 Flash foi instanciado para execução da tarefa. Com base nas credenciais, utilizei o Service Account para configurar o provedor e instanciar o modelo e o agente.

In [ ]:
# Configurando a service account através do JSON fornecido
credentials = service_account.Credentials.from_service_account_file(
    'dados/charla-cuore-norma-278aadacc1b2.json',
    scopes=['https://www.googleapis.com/auth/cloud-platform'],
)

# Instanciando o provider, modelo e agente
provider = GoogleProvider(credentials=credentials, project='charla-cuore-norma')
model = GoogleModel('gemini-2.5-flash', provider=provider)
agent = Agent(model)

Utilizando o agente instanciado, prossegui para a execução da extração das informações de interesse. Aqui, optei por utilizar o módulo assíncrono devido à sua eficiência no contexto de execução dentro de um Jupyter Notebook.

O prompt utilizado seguiu o modelo "task-specific", iniciando com a descrição de uma persona e a tarefa a ser realizada. Esse tipo de prompt foi escolhido por sua capacidade de direcionar o modelo para uma tarefa específica, garantindo que a saída seja consistente e contenha apenas os campos de interesse. É importante ressaltar que esse prompt não é exaustivo e, em um ambiente real, poderia ser discutido com a equipe para torná-lo mais robusta e flexível a diferentes estruturas de arquivos.

A lista de arquivos PDFs obtida na etapa 2 foi utilizada. Para cada arquivo, iterei sobre os caminhos na lista para executar a extração. Além disso, adicionei uma estrutura de logging para armazenar informações sobre a execução e possíveis erros, pensando em um cenário real, mesmo que erros não fossem esperados neste exemplo.

In [ ]:
async def extrair_notas_fiscais(agent, lista_pdfs):
    # Inicializando listas para armazenar resultados e erros
    resultados = []
    erros = []
    
    # Iterando sobre cada arquivo PDF na lista fornecida
    for pdf_path in lista_pdfs:
        try:
            # Executando o agente para extrair informações estruturadas do PDF
            resultado = await agent.run([
                "Você é um assistente especializado em extrair informações estruturadas de Notas Fiscais de Serviço em PDF. "
                "Sua resposta deve ser exclusivamente um objeto JSON puro, sem formatação adicional, comentários ou explicações. "
                "Certifique-se de que a saída siga rigorosamente o modelo ExtracaoOutput e contenha os campos:"
                "descricao_servico, valor_servico, numero_nota, data_emissao, valor_total, cnpj_prestador.",
                
                # Indicando o conteúdo para leitura
                BinaryContent(data=Path(pdf_path).read_bytes(), media_type="application/pdf"),
                
                # Indicando a estrutura de saída esperada
                "ExtracaoOutput"
            ])
            # Adicionando o resultado à lista de resultados
            resultados.append({'arquivo': pdf_path, 'output': resultado.output})
            logging.info(f'Sucesso: {pdf_path}')
        
        except Exception as e:
            # Registrando possíveis erros
            erros.append({'arquivo': pdf_path, 'erro': str(e)})
            logging.error(f'Erro em {pdf_path}: {e}')
    
    # Retornando os resultados e erros
    return resultados, erros

### 5. Output

Após a execução da extração, os resultados obtidos precisaram ser tratados para garantir que estivessem no formato correto de JSON. Embora o prompt tenha sido ajustado para solicitar uma saída consistente, o modelo pode retornar respostas com variações, como delimitadores de bloco de código ou formatações adicionais. 

Para lidar com essas inconsistências, foi necessário implementar uma etapa de pós-processamento para remover possiveis elementos indesejados e garantir que o JSON seja válido e utilizável. 

Acredito que em um ambiente de produção, o pós-processamento seria uma etapa crítica para assegurar que os dados estejam no formato esperado, independentemente de variações na saída do modelo. Além disso, essa abordagem permite maior flexibilidade para lidar com diferentes tipos de entradas e possíveis ajustes futuros no comportamento do modelo.

In [15]:
# Extraindo os resultados
resultados, erros = await extrair_notas_fiscais(agent, lista_pdfs)

# Iniciando a lista para armazenar os resultados pós-processados
resultados_tratados = []

# Exibindo a saída para cada arquivo, em formato JSON
print("Saídas dos resultados:\n")
for res in resultados:
    try:
        # Se necessário, convertendo o campo 'output' de string para JSON
        output_json = json.loads(res['output'].strip('```json').strip())

        # Adicionando o resultado diretamente à lista de resultados tratados
        resultados_tratados.append({"arquivo": res['arquivo'], "output": output_json})

        # Exibindo o nome do arquivo seguido pelo JSON
        print(f"Arquivo: {res['arquivo']}\n")
        print(json.dumps(output_json, indent=4, ensure_ascii=False) + "\n")

    except json.JSONDecodeError:
        # Caso o campo 'output' não seja um JSON válido, registrar o erro
        logging.error(f"Erro ao processar o arquivo: {res['arquivo']}")
        print(f"Arquivo: {res['arquivo']}")
        print(json.dumps(res['output'], indent=4, ensure_ascii=False))

Saídas dos resultados:

Arquivo: dados\nf_paulo_rohan.pdf

{
    "descricao_servico": "CAP AXXIS AB METRO S SOLID A2 TIT",
    "valor_servico": 479.99,
    "numero_nota": "000006950",
    "data_emissao": "27/02/2025",
    "valor_total": 479.99,
    "cnpj_prestador": "22113749000188"
}

Arquivo: dados\nf_paulo_rohan_2.pdf

{
    "descricao_servico": "C.JNS RETA ESP ESCURA BLACK 44",
    "valor_servico": "199,90",
    "numero_nota": "000327907",
    "data_emissao": "25/02/2024",
    "valor_total": "199,90",
    "cnpj_prestador": "02.737.654/0008-02"
}

